<a href="https://colab.research.google.com/github/SBGlab/GoldenStandardDataset/blob/main/create_sample_info_file_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Import Golden Standard Dataset folder from GitHub

!git clone https://github.com/SBGlab/GoldenStandardDataset.git

Cloning into 'GoldenStandardDataset'...
remote: Enumerating objects: 251, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 251 (delta 29), reused 2 (delta 2), pack-reused 196 (from 1)
Receiving objects: 100% (251/251), 11.16 MiB | 9.64 MiB/s, done.
Resolving deltas: 100% (102/102), done.


In [2]:
#@title Import modules and functions

import sys
sys.path.append('/content/GoldenStandardDataset')

import ipywidgets as widgets
from ipywidgets import Layout
from google.colab import files as FILES

from functions import *

print('Modules and functions imported successfully')

Modules and functions imported successfully


# Sample well structure

In [3]:
#@title Select plate type

plate_type = widgets.RadioButtons(options=['96-well plate', '384-well plate'])
display(plate_type)

RadioButtons(options=('96-well plate', '384-well plate'), value='96-well plate')

In [4]:
#@title Enter number of samples

if plate_type.value == '96-well plate':
  wells = wells_in_plate_96
else:
  wells = wells_in_plate_384

style = {'description_width': 'initial'} #for widgets layout
n_s = widgets.IntSlider(
    min=1,
    max=40,
    step=1,
    description='Number of samples:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style = style,
    layout = {'width': '600px'}
)
display(n_s)

each_well = widgets.Checkbox(description='No samples, analyze each well separately', style=style)
display(each_well)

IntSlider(value=1, continuous_update=False, description='Number of samples:', layout=Layout(width='600px'), ma…

Checkbox(value=False, description='No samples, analyze each well separately', style=DescriptionStyle(descripti…

In [5]:
#@title Enter sample names (avoid empty and duplicate names)

if each_well.value == False:
  n_samples = []
  for i in range(n_s.value):
      n_samples.append('Sample_' + str(i+1))
  n_samples

  sample_widget = []
  for tube in n_samples:
      item = widgets.Text(
      placeholder='Sample name',
      description=tube,
      disabled=False)
      display(item)
      sample_widget.append(item)

else:
  print('Every well will be analyzed separately')

Text(value='', description='Sample_1', placeholder='Sample name')

Text(value='', description='Sample_2', placeholder='Sample name')

Text(value='', description='Sample_3', placeholder='Sample name')

Text(value='', description='Sample_4', placeholder='Sample name')

Text(value='', description='Sample_5', placeholder='Sample name')

Text(value='', description='Sample_6', placeholder='Sample name')

In [ ]:
#@title Create Sample-Well structure

if each_well.value == False:
  sample_names = [s.value for s in sample_widget]

  l_test = []
  for i, s in enumerate(sample_names):
    l_test.append(s)
    samp = 'Sample_' + str(i+1)
    if s == '':
      raise TypeError('%s name is empty' %samp)
    if check_unique(l_test) == True:
        raise TypeError('%s name "%s" is duplicate, names must be unique' %(samp, s))

  print('\nDisplaying layout...')

  items = []
  for sample in sample_names:
      item = [widgets.Checkbox(disabled=False, indent=False, description=i) for i in wells]
      items.append(item)

  p = [widgets.GridBox(item, layout=widgets.Layout(grid_template_columns="repeat(12, 50px)")) for item in items]
  children = p

  print('\nSelect wells  for each sample: ')

  tab = widgets.Tab()
  tab.children = children
  [tab.set_title(i, title) for i, title in enumerate(sample_names)]
  display(tab)

  samples = []
  for sample, item in zip(sample_names, items):
      samples.append((sample, item))

### this widget creates a representation of the 96-well plate to allow the user select each well corresponding to different samples
#this is interactive, so we can change it later. Then update in the next command

blanks = []
blank_names = 'blank'

print('\n Select wells for blank')

items = [widgets.Checkbox(disabled=False, indent=False, description=i) for i in wells]
b = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(12, 60px)"))
display(b)

blanks.append(('blank', items))

In [ ]:
#@title Confirm sample selection

%%capture sample_txt
#get the values for each well and associate it with the corresponding sample
samples_def = {} #dict to contain sample name and associated wells
blanks_def = {} #dicto to contain wells associated with blank
wells_def = [] #list to keep track of wells used in the plate

#create samples dict
for sample in samples:
    checkbox_obj = sample[1]
    samples_temp = []
    for well in checkbox_obj:
        check = well.value
        check_well = well.description
        if check == True:
            samples_temp.append(check_well)
            wells_def.append(check_well) #update used wells info
    samples_def.update({sample[0]:samples_temp})
#create blanks info
for blk in blanks:
    checkbox_obj = blk[1]
    blk_temp = []
    for well in checkbox_obj:
        check = well.value
        check_well = well.description
        if check == True:
            blk_temp.append(check_well)
            wells_def.append(check_well) #update used wells info
    blanks_def.update({blk[0] : blk_temp})

#capture print output to generate .txt with sample and blank info
print('Samples:\n')
print(*samples_def.items(), sep='\n')
print('\n\nBlanks:\n')
print(*blanks_def.items(), sep='\n')

In [ ]:
#@title Print sample output

print('Samples:\n')
print(*samples_def.items(), sep='\n')
print('\n\nBlanks:\n')
print(*blanks_def.items(), sep='\n')

In [ ]:
#@title Sample file export

download_button = widgets.Button(description='Download')

def click_download_button(b):
  #clear_output(wait=True)
  #display(button)
  if fname.value == '':
    print('Enter a name for the output file')
  else:
    temp = create_file(fname.value + '_SAMPLES', format='.txt')
    f = open(temp, 'w')
    f.write(str(sample_txt))
    f.close()
    FILES.download(temp)

download_button.on_click(click_download_button)

style = {'description_width': 'initial'}
fname = widgets.Text(
    placeholder='e.g. my_experiment_1',
    description='Enter file name:',
    style=style,
    disabled=False)

items = [fname, download_button]
grid = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns='350px 350px'))
display(grid)

GridBox(children=(Text(value='', description='Enter file name:', placeholder='e.g. my_experiment_1', style=Des…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>